# Estimating Cargo
This example explores how you can use Vektorius to perform more comlex analysis to estimate soybean commodity flows out of Brazil.  

In particular we look at

  1. using Vektorius to perform data preparation and filtering
  2. joining `vessels` and `voyages` data to estimate cargo
  3. comparing our results to published statistics¶

In [ ]:
import warnings
warnings.simplefilter('ignore')

import datetime
import json
import os

from functools import wraps

import ibis
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from shapely.geometry import MultiPolygon, shape
from descarteslabs.vektorius import vector
import displacement

%matplotlib inline

In [ ]:
# helper functions
def str_to_dt(s):
    year, month = s.split("_")
    return datetime.datetime(int(year), int(month), 1)

def displacement_helper(dwt, draft, length, width, return_val):
    klass, Cb, Td, disp = displacement.get_displacement(dwt, draft, length, width)
    if return_val == "disp":
        return disp
    elif return_val == "klass":
        return klass

def cached_berth(f):
    memo = dict()
    @wraps(f)
    def wrapped(point, shape_dict):
        hashable = point.wkt
        if hashable in memo:
            return memo[hashable]
        result = f(point, shape_dict)
        memo[hashable] = result
        return result
    return wrapped

@cached_berth
def get_berth_from_point(point, shape_dict):
    for aoi_name, aoi_poly in shape_dict.items():
        if point.within(aoi_poly):
            return aoi_name
    return np.nan

def get_port_from_berth(s):
    return s.split("_")[0]

# Construct Vessels Query
We're interested in enriching each voyage with information about the ship making the voyage.  To that end we need to locate the Cargo ships that make voyages, and ensure that we have information about the capacity and dimensions of each ship.  Unfortunately, not all vessels have the correct deadweight tonnage (`dwt`) information available, so we want to create some estimated dwt values for vessels that are missing that information.

###### Filter vessels for the right kind of ship

In [ ]:
# Constants for queries:
ship_type = "Cargo"
# used for similar-size calculation
length_thresh = 15
width_thresh = 5

# find all cargo ships
vessels_table = vector.table("vessels")
vessels_query = vessels_table[
    vessels_table.mmsi,
    vessels_table.length,
    vessels_table.width,
    vessels_table.ship_type,
    # capacity can be in either or both, prefer capacity.dwt if present
    vessels_table.capacity.dwt.coalesce(vessels_table.derived_dwt).name("dwt")
]
vessels_query = vessels_query.filter((
    vessels_query.ship_type == ship_type) &
    # length and width must be present to estimate displacement
    vessels_query.length.notnull() &
    vessels_query.width.notnull()
)

###### Estimate dwt for ships missing the value

In [ ]:
# some cargo ships don't have a listed capacity (`dwt`), so we
# need to estimate this value using other cargo ships that 
# have similar dimensions AND have a listed capacity.
# we'll use a self join on the vessels table to find these other
# ships and calculate a mean value for each ship missing capacity
similar_vessels_table = vessels_table.view()
similar_vessels = similar_vessels_table[
    similar_vessels_table.length,
    similar_vessels_table.width,
    similar_vessels_table.ship_type,
    # capacity can be in either or both, prefer capacity.dwt if present
    similar_vessels_table.capacity.dwt.coalesce(similar_vessels_table.derived_dwt).name("dwt")
]
# other ships must have dimensions and capacity to be valid for this calculation.
# Making an assumption that similar sized Passenger Ships are not valid comparisons
# for dwt, since they're designed for different "cargo"
similar_vessels = similar_vessels.filter(similar_vessels.dwt.notnull() &
                                         similar_vessels.length.notnull() & 
                                         similar_vessels.width.notnull() &
                                         (similar_vessels.ship_type == ship_type))

# join the target list of vessels to the list of similar vessels
# and exclude target vessels that already have a capacity, we don't
# need to bother estimating their capacity
vessels_missing_dwt = vessels_query.filter(vessels_query.dwt.isnull())
joined = vessels_missing_dwt.cross_join(similar_vessels)
joined = joined[
    vessels_missing_dwt.mmsi,
    vessels_missing_dwt.length,
    vessels_missing_dwt.width,
    vessels_missing_dwt.ship_type,
    similar_vessels.dwt,
    similar_vessels.length.name("similar_length"),
    similar_vessels.width.name("similar_width")
]
joined.filter(
    # length and width of similar ships is +/- some threshhold value
    (joined.similar_length.between(joined.length - length_thresh, joined.length + length_thresh)) &
    (joined.similar_width.between(joined.width - width_thresh, joined.width + width_thresh))
)
    
# calculate mean value of similar ships for each ship with unknown capacity
estimated_dwt_vessels = joined.group_by([
    joined.mmsi,
    joined.length,
    joined.width,
    joined.ship_type,
]).aggregate(dwt=joined.dwt.mean())

###### Create the final vessels query

In [ ]:
# create the final list of vessels with listed and estimated capacity
final_vessels = vessels_query.left_join(estimated_dwt_vessels,
                                        vessels_query.mmsi == estimated_dwt_vessels.mmsi)
final_vessels = final_vessels[
    vessels_query.mmsi,
    vessels_query.length,
    vessels_query.width,
    vessels_query.ship_type,
    vessels_query.dwt.coalesce(estimated_dwt_vessels.dwt).name("dwt"),
    # allows us to determine if the dwt was estimated or listed
    estimated_dwt_vessels.mmsi.isnull().name("estimated")
]
# if there still isn't a listed capcity, then remove those ships
final_vessels = final_vessels.filter(final_vessels.dwt.notnull() &
                                     (final_vessels.dwt != 0))

# Construct Voyages Query
Now that we've identified potential cargo ships, we can find the voyages those ships have made and add the vessel info to the voyage info.

In [ ]:
# load brazilian grain berth shapes and make a multipolygon to query from voyages
workdir = os.getcwd()
fname = os.path.join(workdir, "brazil_grain-berths.geojson")

# berth names follow the convention {port}_{berth}
with open(fname) as f:
    feature_collection = json.load(f)

polygons = [shape(f["geometry"]) for f in feature_collection["features"]]
aoi = MultiPolygon(polygons)

voyages_table = vector.table("voyages")

departure_filter = voyages_table.departure.between("2017-04-01", "2020-03-31")
aoi_filter = voyages_table.origin.within(aoi)
voyages_query = voyages_table.filter(departure_filter & aoi_filter)

final_voyages = final_vessels.inner_join(voyages_query, final_vessels.mmsi == voyages_query.mmsi)
final_voyages = final_voyages[
    final_vessels.mmsi,
    final_vessels.length,
    final_vessels.width,
    final_vessels.dwt,
    final_vessels.ship_type,
    voyages_query.departure,
    voyages_query.arrival,
    voyages_query.origin,
    voyages_query.destination,
    voyages_query.avg_draft,
    voyages_query.avg_speed,
    voyages_query.est_cargo,
    voyages_query.departure.month().name("month"),
    voyages_query.departure.year().name("year")
]

vdf = final_voyages.execute()

Now that we finally have a DataFrame, we can perform some extra data prep that's easier to do with Pandas

In [ ]:
# fill important missing values with average of valid ones
vdf.loc[vdf["avg_draft"].isin([np.nan, np.inf, -np.inf]), "avg_draft"] = \
    vdf.loc[~vdf["avg_draft"].isin([np.nan, np.inf, -np.inf]), "avg_draft"].mean()

vdf.loc[vdf["avg_speed"].isin([np.nan, np.inf, -np.inf]), "avg_speed"] = \
    vdf.loc[~vdf["avg_speed"].isin([np.nan, np.inf, -np.inf]), "avg_speed"].mean()

assert(vdf.loc[vdf["avg_draft"].isin([np.nan, np.inf, -np.inf]), "avg_draft"].shape[0] == 0)
assert(vdf.loc[vdf["avg_speed"].isin([np.nan, np.inf, -np.inf]), "avg_speed"].shape[0] == 0)

# extra fields for analysis
vdf["year_month"] = vdf["year"].astype(str) + "_" + vdf["month"].astype(str)

assert(vdf.loc[vdf["dwt"].isin([np.nan, np.inf, -np.inf]), "dwt"].shape[0] == 0)

# Calculate cargo
Finally we've collected all the information we need to estimate cargo:

In [ ]:
# calculate displacement and vessel class
vdf["est_cargo"] = vdf.apply(lambda x: displacement_helper(x.dwt, x.avg_draft, x.length, x.width, "disp"), axis=1)
vdf["class"] = vdf.apply(lambda x: displacement_helper(x.dwt, x.avg_draft, x.length, x.width, "klass"), axis=1)

# Drop negative displacements (still under investigation)
print(vdf.shape)
vdf = vdf.loc[vdf["est_cargo"] >= 0]
print(vdf.shape)

In [ ]:
# take a peek at the results
vdf.head(3)

# Compare to Brazilian Soybean Exports by Port
Now we can compare our estimated value to the actual statistics published at https://anec.com.br/pt-br/servicos/estatisticas/category/2020-7. To make things a little easier we've put the statistics into an easy to read CSV.

In [ ]:
# load a prepared version of the published version of the statistics
fname = os.path.join(workdir, "anec_soybean_port_exports_201701-202006.csv")
apdf = pd.read_csv(fname)

# add a timestamp corresponding to the actual month data
apdf["date"] = (apdf["year"].astype(str) + "_" + apdf["month"].astype(str)).apply(str_to_dt)
apdf = apdf[["date", "volume", "port"]]

apdf.head(2)

### Tag origin port in existing voyages

In [ ]:
vdf = vdf.copy()

# first make a dictionary of shapely shapes for each berth
aoi_polys = {feat["properties"]["name"]: shape(feat["geometry"])
             for feat in feature_collection["features"]}

# identify berth of origin and identify port as well - # berth names follow the convention {port}_{berth}
vdf["berth"] = vdf.apply(lambda x: get_berth_from_point(x.origin, aoi_polys), axis=1)
vdf["port"] = vdf["berth"].apply(get_port_from_berth)

assert((vdf.loc[vdf["berth"].isna()].shape[0] == 0) & (vdf.loc[vdf["port"].isna()].shape[0] == 0))

### Compare for one port

In [ ]:
# Look at Santos port as an example
uport = "santos"

# isolate the export data for this port
apdf_tmp = apdf[apdf["port"] == uport].copy()
apdf_tmp= apdf_tmp.set_index("date")

# isolate the voyage data for this port and group by month
vdf_tmp = vdf[vdf["port"] == uport].copy().groupby("year_month").sum()[["est_cargo"]].reset_index()
vdf_tmp["date"] = vdf_tmp["year_month"].apply(str_to_dt)
vdf_tmp = vdf_tmp.sort_values(by="date").drop("year_month", axis=1).set_index("date")

# merge the two
merge_tmp = vdf_tmp.merge(apdf_tmp, how='outer', left_index=True, right_index=True).dropna()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,5))

# time series comparison
ax[0].plot(merge_tmp.index, merge_tmp["est_cargo"] / 1e3, '-o', label="voyages_cargo")
ax[0].plot(merge_tmp.index, merge_tmp["volume"] / 1e3, '-o', label="anec_volume")
ax[0].set_title(uport, fontsize=15)
ax[0].grid(True)
ax[0].legend(loc="upper left")
ax[0].set_ylabel("Thousand Tons")

# scatter plot comparison
ax[1].scatter(merge_tmp["est_cargo"].values, merge_tmp["volume"].values)
ax[1].grid(True)
ax[1].set_ylabel("anec volume (1000 tons)")
ax[1].set_xlabel("voyages displacement (1000 tons)");

# Filter voyages ending in China
There is some correlation present, but not enough to draw a meaningful relationship. More than 80% of Brazil's soybean exports go to China, filtering for voyages ending in China should tighten up the signal

In [ ]:
china_coast_fc = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {"name": "china_coast"},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [[120.498046875, 41.77131167976407],[117.42187500000001, 39.842286020743394],[116.3671875, 37.579412513438385],[118.38867187500001, 35.10193405724606],[118.38867187500001, 31.952162238024975],[117.158203125, 29.53522956294847],[113.90625, 26.980828590472107],[110.91796875, 25.403584973186703],[107.314453125, 23.725011735951796],[106.61132812499999, 23.079731762449878],[108.06152343749999, 21.49396356306447],[109.423828125, 20.838277806058933],[111.796875, 20.715015145512087],[114.697265625, 21.3303150734318],[117.50976562499999, 22.59372606392931],[119.17968749999999, 24.407137917727667],[121.5087890625, 27.176469131898898],[122.6953125, 29.99300228455108],[123.26660156249999, 31.466153715024294],[121.904296875, 33.90689555128866],[121.06933593749999, 35.639441068973944],[123.31054687499999, 36.70365959719456],[124.1455078125, 38.09998264736481],[124.23339843749999, 40.07807142745009],[122.607421875, 41.244772343082076],[120.498046875, 41.77131167976407]
          ]]
      }
    }
  ]
}

# make a dict of shapely shapes
china_coast_polys = {feat["properties"]["name"]: shape(feat["geometry"])
                     for feat in china_coast_fc["features"]}

# identify voyages that ended in China and drop all others
vdf["china_dest"] = vdf.apply(lambda x: get_berth_from_point(x.destination, china_coast_polys), axis=1)

print(vdf.shape)
vdf = vdf[~vdf["china_dest"].isna()]
print(vdf.shape)

In [ ]:
# re-examine Santos port
uport = "santos"

# isolate the export data for this port
apdf_tmp = apdf[apdf["port"] == uport].copy()
apdf_tmp= apdf_tmp.set_index("date")

# isolate the voyage data for this port and group by month
vdf_tmp = vdf[vdf["port"] == uport].copy().groupby("year_month").sum()[["est_cargo"]].reset_index()
vdf_tmp["date"] = vdf_tmp["year_month"].apply(str_to_dt)
vdf_tmp = vdf_tmp.sort_values(by="date").drop("year_month", axis=1).set_index("date")

# merge the two
merge_tmp = vdf_tmp.merge(apdf_tmp, how='outer', left_index=True, right_index=True).dropna()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,5))

# time series comparison
ax[0].plot(merge_tmp.index, merge_tmp["est_cargo"] / 1e3, '-o', label="voyages_cargo")
ax[0].plot(merge_tmp.index, merge_tmp["volume"] / 1e3, '-o', label="anec_volume")
ax[0].set_title(uport, fontsize=15)
ax[0].grid(True)
ax[0].legend(loc="upper left")
ax[0].set_ylabel("Thousand Tons")

# scatter plot comparison
ax[1].scatter(merge_tmp["est_cargo"].values, merge_tmp["volume"].values)
ax[1].grid(True)
ax[1].set_ylabel("anec volume (1000 tons)")
ax[1].set_xlabel("voyages displacement (1000 tons)");

The estimated volume magnitude is off, signaling some improvements needed in cargo calculation, but the correlation is much tighter